# Báo Cáo Chiến Lược Thực Thi Pipeline Các Thuật Toán Sentiment Analysis

## Mục Lục
1. [Tổng Quan Project](#1-tổng-quan-project)
2. [Kiến Trúc Pipeline](#2-kiến-trúc-pipeline)
3. [Chiến Lược Xử Lý Dữ Liệu](#3-chiến-lược-xử-lý-dữ-liệu)
4. [So Sánh Các Thuật Toán](#4-so-sánh-các-thuật-toán)
4. [Cấu Hình Tối Ưu](#5-cấu-hình-tối-ưu)
6. [Kết Quả Thực Nghiệm](#6-kết-quả-thực-nghiệm)
7. [Khuyến Nghị](#7-khuyến-nghị)

---

## 1. Tổng Quan Project

Project này thực hiện **Sentiment Analysis** trên dataset Amazon Reviews với mục tiêu:
- Phân loại sentiment (positive/negative) của reviews
- So sánh hiệu suất của 3 thuật toán Machine Learning
- Tối ưu hóa pipeline để đạt độ chính xác cao nhất

### Đặc điểm Dataset
- **Nguồn**: Amazon Review Polarity Dataset từ Kaggle
- **Kích thước**: Có thể điều chỉnh (1000-40000 samples training)
- **Định dạng**: CSV với cột text và label (1: positive, 2: negative)
- **Thách thức**: Text không cấu trúc, nhiễu, độ dài khác nhau

## 2. Kiến Trúc Pipeline

### 2.1 Workflow Tổng Thể

```
┌─────────────────┐    ┌──────────────────┐    ┌─────────────────┐
│   Data Loading  │───▶│  Preprocessing   │───▶│  Feature        │
│   (Kaggle/Local)│    │  (Text Cleaning) │    │  Extraction     │
└─────────────────┘    └──────────────────┘    │  (TF-IDF)       │
                                               └─────────────────┘
                                                         │
┌─────────────────┐    ┌──────────────────┐    ┌─────────────────┐
│   Model         │◀───│   Model          │◀───│   Model         │
│   Evaluation    │    │   Training       │    │   Selection     │
└─────────────────┘    └──────────────────┘    └─────────────────┘
```

### 2.2 Các Thành Phần Chính

| Component | File | Chức năng |
|-----------|------|-----------|
| **Data Loader** | `kaggle_data_loader.py`, `local_data_loader.py` | Tải và chuẩn bị dữ liệu |
| **Preprocessor** | `pre_processor.py` | Làm sạch và chuẩn hóa text |
| **Feature Extractor** | `tf_idf_vectorizer.py` | Chuyển đổi text thành vector |
| **Model Trainers** | `*_classifier.py` | Huấn luyện các thuật toán |
| **Pipeline Controller** | `main_modal_traning_focus.py` | Điều phối toàn bộ quá trình |
| **Config Manager** | `config_loader.py` | Quản lý cấu hình |

## 3. Chiến Lược Xử Lý Dữ Liệu

### 3.1 Text Preprocessing Strategy

**Mục tiêu**: Tối ưu hóa chất lượng text để cải thiện hiệu suất model

#### 3.1.1 Quy Trình Preprocessing
```python
# Workflow trong PreProcessor.preprocess_for_sentiment()
Text Input → Text Cleaning → Tokenization → Stopword Removal → Lemmatization → Normalization
```

#### 3.1.2 Các Kỹ Thuật Áp Dụng

| Bước | Kỹ thuật | Lý do |
|------|----------|-------|
| **Text Cleaning** | Loại bỏ HTML, URL, ký tự đặc biệt | Giảm nhiễu, chuẩn hóa input |
| **Preserve Negation** | Giữ lại "not", "no", "never" | Quan trọng cho sentiment analysis |
| **Lemmatization** | Đưa về dạng gốc của từ | Giảm số lượng features, tăng tính tổng quát |
| **Lowercase** | Chuyển về chữ thường | Tránh phân biệt chữ hoa/thường |
| **Remove Duplicates** | Loại bỏ text trùng lặp | Tránh overfitting |

#### 3.1.3 Sentiment-Aware Features
- **Preserving Negation**: Đặc biệt quan trọng cho sentiment
- **Custom Stopwords**: Loại bỏ từ không mang ý nghĩa sentiment
- **Text Length Control**: Xử lý reviews quá ngắn/dài

## 4. Cấu Hình Tối Ưu

### 4.1 Configuration Strategy

Project sử dụng **JSON-based configuration system** để:
- Dễ dàng thay đổi hyperparameters
- So sánh multiple configurations
- Reproducible experiments
- Parallel testing của different strategies

### 4.2 Các Loại Configuration

#### 4.2.1 Balanced Configuration
- **Mục tiêu**: Cân bằng giữa accuracy và training time
- **Dataset size**: Medium (5000-10000 samples)
- **Training time**: 5-10 phút
- **Use case**: Development và testing

#### 4.2.2 Accuracy Optimized Configuration  
- **Mục tiêu**: Đạt accuracy cao nhất có thể
- **Dataset size**: Large (20000+ samples)
- **Training time**: 15-30 phút
- **Use case**: Final model cho production

#### 4.2.3 Speed Optimized Configuration
- **Mục tiêu**: Training nhanh nhất
- **Dataset size**: Small (1000-2000 samples)
- **Training time**: 1-3 phút
- **Use case**: Rapid prototyping


## 4.3 Chi Tiết Hyperparameters cho Từng Thuật Toán

### 4.3.1 Logistic Regression Parameters

#### Core Parameters
```python
LogisticRegression(
    C=1.0,                    # Regularization strength (inverse)
    solver='liblinear',       # Optimization algorithm
    max_iter=1000,           # Maximum iterations
    random_state=42          # For reproducibility
)
```

**Giải thích chi tiết:**

| Parameter | Ý nghĩa | Giá trị khuyến nghị | Tác động |
|-----------|---------|-------------------|----------|
| **C** | Độ mạnh regularization (nghịch đảo) | 0.1 - 10.0 | • C nhỏ: Regularization mạnh, tránh overfitting<br>• C lớn: Ít regularization, có thể overfit |
| **solver** | Thuật toán tối ưu hóa | 'liblinear', 'lbfgs' | • 'liblinear': Tốt cho small dataset<br>• 'lbfgs': Tốt cho large dataset |
| **max_iter** | Số lần lặp tối đa | 1000-5000 | • Quá thấp: Model chưa converge<br>• Quá cao: Tốn thời gian training |
| **penalty** | Loại regularization | 'l1', 'l2' | • L1: Feature selection<br>• L2: Shrink coefficients |

#### Advanced Parameters
```python
# Thêm các parameters nâng cao
LogisticRegression(
    penalty='l2',             # L1/L2 regularization
    class_weight='balanced',  # Handle imbalanced data
    dual=False,              # Primal vs dual formulation
    fit_intercept=True       # Add bias term
)
```

### 4.3.2 Random Forest Parameters

#### Core Parameters
```python
RandomForestClassifier(
    n_estimators=100,        # Số cây trong forest
    max_depth=None,          # Độ sâu tối đa của cây
    min_samples_split=2,     # Số samples tối thiểu để split
    min_samples_leaf=1,      # Số samples tối thiểu ở leaf
    random_state=42
)
```

**Giải thích chi tiết:**

| Parameter | Ý nghĩa | Giá trị khuyến nghị | Tác động |
|-----------|---------|-------------------|----------|
| **n_estimators** | Số cây trong ensemble | 50-200 | • Nhiều cây: Accuracy cao hơn, training lâu hơn<br>• Ít cây: Training nhanh, có thể underfit |
| **max_depth** | Độ sâu tối đa của cây | 10-20 hoặc None | • Sâu: Có thể overfit<br>• Nông: Có thể underfit<br>• None: Cây phát triển tự nhiên |
| **min_samples_split** | Min samples để split node | 2-10 | • Thấp: Cây chi tiết hơn, có thể overfit<br>• Cao: Cây đơn giản hơn, tránh overfit |
| **min_samples_leaf** | Min samples ở leaf node | 1-5 | • Thấp: Leaf có ít samples, chi tiết<br>• Cao: Leaf có nhiều samples, tổng quát |

#### Advanced Parameters
```python
RandomForestClassifier(
    max_features='sqrt',      # Số features random cho mỗi split
    bootstrap=True,          # Bootstrap sampling
    oob_score=True,          # Out-of-bag score
    class_weight='balanced', # Handle imbalanced classes
    criterion='gini'         # Splitting criterion
)
```

**Advanced Parameters Explained:**

| Parameter | Ý nghĩa | Options | Tác động |
|-----------|---------|---------|----------|
| **max_features** | Features xem xét cho mỗi split | 'sqrt', 'log2', int, float | • 'sqrt': √n_features, good default<br>• 'log2': log₂(n_features)<br>• int: Exact number |
| **criterion** | Hàm đo độ không thuần khiết | 'gini', 'entropy' | • 'gini': Nhanh hơn<br>• 'entropy': Information gain |
| **bootstrap** | Sampling với replacement | True/False | • True: Diversity trong training<br>• False: Sử dụng toàn bộ dataset |

### 4.3.3 Gradient Boosting Parameters

#### Core Parameters
```python
GradientBoostingClassifier(
    n_estimators=100,        # Số boosting stages
    learning_rate=0.1,       # Tốc độ học
    max_depth=3,            # Độ sâu của weak learners
    subsample=1.0,          # Fraction samples cho training
    random_state=42
)
```

**Giải thích chi tiết:**

| Parameter | Ý nghĩa | Giá trị khuyến nghị | Tác động |
|-----------|---------|-------------------|----------|
| **n_estimators** | Số boosting rounds | 50-200 | • Nhiều: Có thể overfit nhưng accuracy cao<br>• Ít: Có thể underfit |
| **learning_rate** | Shrinkage rate | 0.01-0.3 | • Thấp: Training chậm, cần nhiều estimators<br>• Cao: Training nhanh, dễ overfit |
| **max_depth** | Độ sâu weak learner | 3-6 | • Shallow: Simple learners, tránh overfit<br>• Deep: Complex learners, có thể overfit |
| **subsample** | Tỷ lệ samples training | 0.5-1.0 | • < 1.0: Stochastic gradient boosting<br>• = 1.0: Deterministic boosting |

#### Advanced Parameters
```python
GradientBoostingClassifier(
    loss='deviance',         # Loss function
    min_samples_split=2,     # Min samples to split
    min_samples_leaf=1,      # Min samples at leaf
    max_features=None,       # Features to consider
    validation_fraction=0.1, # Fraction for early stopping
    n_iter_no_change=5      # Early stopping patience
)
```

**Advanced Parameters Explained:**

| Parameter | Ý nghĩa | Options | Tác động |
|-----------|---------|---------|----------|
| **loss** | Hàm loss | 'deviance', 'exponential' | • 'deviance': Logistic regression loss<br>• 'exponential': AdaBoost loss |
| **validation_fraction** | Data cho early stopping | 0.1-0.2 | • Monitor overfitting<br>• Automatic stopping |
| **warm_start** | Incremental training | True/False | • True: Có thể thêm estimators<br>• False: Training từ đầu |

### 4.3.4 TF-IDF Vectorizer Parameters

#### Core Parameters  
```python
TfidfVectorizer(
    max_features=10000,      # Số features tối đa
    ngram_range=(1, 2),      # N-gram range
    min_df=2,               # Min document frequency
    max_df=0.95,            # Max document frequency
    stop_words='english'     # Stopwords removal
)
```